# Libraries

In [ ]:
!pip install ktrain

# GeoTransformers Model

In [ ]:
import folium
from folium.plugins import HeatMap

In [ ]:
import ktrain
from ktrain import text
import numpy as np
import pandas as pd

class GeoTransformers:
   
    def __init__(self):
        self.version=1

    def set_metadata_geo(self,metadata_geo):
        self.metadata_geo=metadata_geo
    
    def download_geomodels(self):
        !gdown --id 13Bra_R4qG2TPxBuk1Toh26NqDu0T_T9q
        !gdown --id 1aKIadFQ8Q9y_uUwBBUgoqdhWfKPw4JG4
    
    def extract_geomodels(self):
        !tar -xzvf geotransformers-v20220918.tar.gz
        !tar -xzvf geobert-e5-level2-geocluster-0.model.tar.gz
        !tar -xzvf geobert-e5-level2-geocluster-1.model.tar.gz
        !tar -xzvf geobert-e5-level2-geocluster-2.model.tar.gz
        !tar -xzvf geobert-e5-level2-geocluster-3.model.tar.gz
        !tar -xzvf geobert-e5-level2-geocluster-4.model.tar.gz
        !tar -xzvf geobert-e5-level2-geocluster-5.model.tar.gz
        !tar -xzvf geobert-e5-level2-geocluster-6.model.tar.gz
        !tar -xzvf geobert-e5-level2-geocluster-7.model.tar.gz
        !tar -xzvf geobert-level0-e9.model.tar.gz
    
    def load_geomodels(self):
        self.predictor_level1 = ktrain.load_predictor('geobert-level0-e9.model')
        self.predictor_level2_0 = ktrain.load_predictor('geobert-e5-level2-geocluster-0.model')
        self.predictor_level2_1 = ktrain.load_predictor('geobert-e5-level2-geocluster-1.model')
        self.predictor_level2_2 = ktrain.load_predictor('geobert-e5-level2-geocluster-2.model')
        self.predictor_level2_3 = ktrain.load_predictor('geobert-e5-level2-geocluster-3.model')
        self.predictor_level2_4 = ktrain.load_predictor('geobert-e5-level2-geocluster-4.model')
        self.predictor_level2_5 = ktrain.load_predictor('geobert-e5-level2-geocluster-5.model')
        self.predictor_level2_6 = ktrain.load_predictor('geobert-e5-level2-geocluster-6.model')
        self.predictor_level2_7 = ktrain.load_predictor('geobert-e5-level2-geocluster-7.model')

    def load_geoclusters(self):
        self.df_geoclusters = pd.read_csv('geoclusters.csv')
    
    def predict(self,input_txt,n=3):
        probs = self.predictor_level1.predict_proba(input_txt)
        df_lv1 = pd.DataFrame(self.predictor_level1.get_classes())
        df_lv1['probs'] = probs
        df_lv1.columns = ['level1_geo','probs']
        L = {}
        for index,row in df_lv1.iterrows():
            L[row['level1_geo']] = self.sensor_map(row['level1_geo'],input_txt,n)
        return df_lv1, L

    def sensor_map(self,lv1,input_txt,n=3):

        predictor_lv2 = self.predictor_level2_0
        if lv1=='level1_geo_0': predictor_lv2 = self.predictor_level2_0
        if lv1=='level1_geo_1': predictor_lv2 = self.predictor_level2_1
        if lv1=='level1_geo_2': predictor_lv2 = self.predictor_level2_2
        if lv1=='level1_geo_3': predictor_lv2 = self.predictor_level2_3
        if lv1=='level1_geo_4': predictor_lv2 = self.predictor_level2_4
        if lv1=='level1_geo_5': predictor_lv2 = self.predictor_level2_5
        if lv1=='level1_geo_6': predictor_lv2 = self.predictor_level2_6
        if lv1=='level1_geo_7': predictor_lv2 = self.predictor_level2_7


        probs_lv2 = predictor_lv2.predict_proba(input_txt)

        df_geoclusters_lv2 = self.df_geoclusters[(self.df_geoclusters.level1_geo==int(lv1.replace('level1_geo_','')))]
        df_geoclusters_lv2['probs'] = probs_lv2


        data_heatmap = []
        for index,row in df_geoclusters_lv2.sort_values(by='probs',ascending=False).head(n).iterrows():
            data_heatmap.append([row['lat'],row['lng'],row['probs']])

        m = folium.Map(zoom_start=2)
        HeatMap(data_heatmap).add_to(m)

        return df_geoclusters_lv2.sort_values(by='probs',ascending=False).head(n), m


In [ ]:
GT = GeoTransformers()
GT.download_geomodels()
GT.extract_geomodels()
GT.load_geomodels()
GT.load_geoclusters()

In [ ]:
s = '''Florida sent 50 migrants to Martha's Vineyard. This is how residents reacted'''
df_results, L = GT.predict(s,n=3)

In [ ]:
df_results

,level1_geo,probs
0,level1_geo_0,0.001585
1,level1_geo_1,0.927484
2,level1_geo_2,0.067379
3,level1_geo_3,0.000316
4,level1_geo_4,0.001781
5,level1_geo_5,0.001435
6,level1_geo_6,0.000003
7,level1_geo_7,0.000018


In [ ]:
L['level1_geo_1'][0]

,level1_geo,level2_geo,lat,lng,probs
126,1,24,27.8333,-81.7170,0.293417
109,1,7,31.1060,-97.6475,0.097143
270,1,168,25.7743,-80.1937,0.087737


In [ ]:
L['level1_geo_1'][1]

In [ ]:
s = '''Florida sent 50 migrants to Martha's Vineyard. This is how residents reacted'''
df_results, map = GT.sensor_map(s,n=5)
map

In [ ]:
s = '''Deforestation rate increased by 52 percent during the years Bolsonaro has been in power.'''
df_results, L = GT.predict(s,n=5)

In [ ]:
df_results

,level1_geo,probs
0,level1_geo_0,0.035855
1,level1_geo_1,0.005952
2,level1_geo_2,0.009523
3,level1_geo_3,0.000283
4,level1_geo_4,0.000237
5,level1_geo_5,0.948041
6,level1_geo_6,0.000089
7,level1_geo_7,0.000020


In [ ]:
L['level1_geo_5'][1]

In [ ]:
s = '''food insecurity, civil wars, attack on human rights'''
df_results, L = GT.predict(s,n=10)
level1_geo = df_results.sort_values(by='probs',ascending=False).head(1).level1_geo.to_list()[0]

In [ ]:
L[level1_geo][1]

In [ ]:
    df_results.sort_values(by='probs',ascending=False).head(1).level1_geo.to_list()[0]

'level1_geo_2'

'level1_geo_0'